In [7]:
#%pip install beautifulsoup4
#%pip install pip-system-certs

In [37]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import stardog
import os
import io
from langchain_community.graphs import Neo4jGraph
from PyPDF2 import PdfReader
from langdetect import detect
import spacy
import re
from unidecode import unidecode

- Buscar os links para a tese no Neo4j
- Buscar o link do PDF no repositório da universidade
- Baixar o PDF
- Extrair o texto do PDF 

Buscar links para a tese no Neo4j

Conectando ao Neo4j 

In [2]:

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

#Connecting to the graph
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [ ]:
# Stardog variables
#STARDOG_ENDPOINT = os.getenv('STARDOG_ENDPOINT')
#STARDOG_USERNAME = os.getenv("STARDOG_USERNAME")
#STARDOG_PASSWORD = os.getenv("STARDOG_PASSWORD")

#connection_details = {
#    'endpoint': STARDOG_ENDPOINT,
#    'username': STARDOG_USERNAME,
#    'password': STARDOG_PASSWORD
#}

In [ ]:
#database_name = 'IndigenousSlavery'
#conn = stardog.Connection(database_name, **connection_details)

In [3]:
#Saprql query
query1 = """
SELECT ?thesis ?link
WHERE {
  ?thesis <http://purl.org/dc/terms/publisher> <tag:stardog:api:USP>.
  ?thesis a <http://purl.org/ontology/bibo/Thesis>.
  ?thesis <http://purl.org/ontology/bibo/repository> ?link.  
}
"""

#csv_results = conn.select(query1, content_type='text/csv')
#thesis_abstract = pd.read_csv(io.BytesIO(csv_results))
#thesis_abstract


#Cypher query 
query2 = """
MATCH  (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:USP'})
RETURN  t["uri"] as Thesis, t["repository"] as repository 
"""

results = graph.query(query2)
# Convert the results to a DataFrame
thesis_repository = pd.DataFrame(results, columns=["Thesis", "repository"])
thesis_repository

#2213 / 2208

,Thesis,repository
0,tag:stardog:api:substituicao_do_negro_de_fumo_...,https://www.teses.usp.br/teses/disponiveis/3/3...
1,tag:stardog:api:diversidade_na_unidade_a_tradi...,https://www.teses.usp.br/teses/disponiveis/71/...
2,tag:stardog:api:kuxima_paa_dizque_antigamente_...,https://www.teses.usp.br/teses/disponiveis/8/8...
3,tag:stardog:api:vai_da_muito_trabalho_cultura_...,https://www.teses.usp.br/teses/disponiveis/48/...
4,tag:stardog:api:access_to_maternal_reproductiv...,https://www.teses.usp.br/teses/disponiveis/22/...
...,...,...
2203,tag:stardog:api:entre_o_tipo_e_o_sujeito_os_re...,http://www.teses.usp.br/teses/disponiveis/27/2...
2204,tag:stardog:api:a_equacao_de_blackscholes_com_...,http://www.teses.usp.br/teses/disponiveis/55/5...
2205,tag:stardog:api:o_feitor_ausente_estudo_sobre_...,https://teses.usp.br/teses/disponiveis/8/8138/...
2206,tag:stardog:api:um_ator_de_fronteira_uma_anali...,http://www.teses.usp.br/teses/disponiveis/27/2...


In [18]:
i = 10
url = thesis_repository['repository'][i]
thesis_uri = thesis_repository['Thesis'][i][16:]

In [19]:
# Funcao para coletar link do pdf

def get_pdf_link(url):
    
    #Fazer requisição e parsear o arquivo html
    f = requests.get(url, verify=True).text#, verify=False) 
    soup = bs(f, "html.parser")
    
    #Coletando link para as teses
    for doc in soup.find_all('a', href=True):
        if doc['href'].endswith('.pdf'):
            path = doc['href']
            prefix_uni = 'https://www.teses.usp.br'
            link = prefix_uni + path

            return link
    
    return None

In [20]:
pdf_link = get_pdf_link(url)

In [23]:
#fazer download do arquivo
def download_file(pdf_link, thesis_uri):
    # NOTE the stream=True parameter below
    r = requests.get(pdf_link, verify=True, stream=True)
    # raise_for_status() is not needed with stream=True
    # r.raise_for_status()
    with open(thesis_uri + '.pdf', 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return thesis_uri

In [24]:
download_file(pdf_link, thesis_uri)

'revisao_narrativa_para_subsidiar_o_entendimento_do_panorama_da_cobertura_vacinal_em_criancas_menores_de_um_ano_de_idade_no_contexto_brasileiro'

Extrair texto do PDF

In [30]:
#Carregando os modelos SpaCy para inglês e português 
nlp_en = spacy.load("en_core_web_lg")
nlp_pt = spacy.load("pt_core_news_lg")

In [31]:
# Funcão que recebe uma string e a limpar para ficar no formato aceitável para uma URI
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))

In [42]:
def extratc_text(tese_path):

    # Processando o arquivo em PDF
    reader = PdfReader(tese_path)
    number_of_pages = len(reader.pages)
    excerpt = {}


    # Iterando por cada página
    for page_number in range(number_of_pages):
        

        # Extraindo o texto da página
        if page_number == 0:
            page_text = (reader.pages[page_number].extract_text() + '\n ' 
                        + reader.pages[page_number+1].extract_text()[:400])
        else:
            
            if page_number == number_of_pages -1:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text())
            
            else:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text() + '\n ' 
                            + reader.pages[page_number+1].extract_text()[:400])  
                
        # Detectando o idioma do texto
        try:
            lang = detect(page_text)
        except:
            lang = None

        # Extraindo as entidades
        if lang == 'pt':
            doc = nlp_pt(page_text)
        if lang == 'en':
            doc = nlp_en(page_text)

        persons = []
        gpes = []
        #orgs = []
        for ent in doc.ents:
            if ent.label_ == "PER": #"PERSON":
                persons.append(process_uri(ent.text))
            if ent.label_ == "LOC": #"GPE":
                gpes.append(process_uri(ent.text))
            #if ent.label_ == "ORG":
            #    orgs.append(process_uri(ent.text))
                
        excerpt[page_number] = {'text': page_text, 
                                'lang': lang, 
                                'persons': list(set(persons)), 
                                'gpes': list(set(gpes)),
                               #'orgs': list(set(orgs))
                               }

    return excerpt


In [43]:
excerpt = extratc_text(thesis_uri + '.pdf')

In [46]:
excerpt[60]

{'text': '59 \n de ESAVIS vem se estabelecendo e ganhando maior dimensão, mas a autonomia na \nprodução de vacinas ainda é menor, em comparação à Índia . Fabricantes de países de \nbaixo e médio nível socioeconômico têm um papel cada vez maior no fornecimento de \nvacinas, e sua a entrada no mercado estimula quedas de preço das principais vacinas .  \n 60 \n 6. DISCUSSÃO  \n \nPelo exposto depreen de-se que, apesar do sucesso dos programas de vacinação, já \nhavia sinais de preocupação em relação à manutenção das conquistas obtidas. A queda \ndas coberturas vacinais vinha ocorrendo principalmente a partir de 2016.  \nA vacinação se caracteriza por interdep endência de responsabilidades individuais, \ncoletivas e institucionais. Esta dissertação priorizou a investigação sobre as \nresponsabilidades institucionais na queda das coberturas vacinais.  \nNesta revisão narrativa sobre a interrelação entre a cobertura vaci nal no Brasil e \noutros indicadores de qualidade do SUS foram encontra

In [ ]:
def extratc_text_NER(tese_uri, tese_path):

    triplas = """ """

    # Processando o arquivo em PDF
    reader = PdfReader(tese_path)
    number_of_pages = len(reader.pages)

    # Iterando por cada página
    for page_number in range(number_of_pages):
        

        # Extraindo o texto da página
        if page_number == 0:
            page_text = (reader.pages[page_number].extract_text() + '\n ' 
                        + reader.pages[page_number+1].extract_text()[:400])
        else:
            
            if page_number == number_of_pages -1:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text())
            
            else:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text() + '\n ' 
                            + reader.pages[page_number+1].extract_text()[:400])  

        # Detectando o idioma do texto
        try:
            lang = detect(page_text)
        except:
            lang = None

        # Extraindo as entidades
        if lang == 'pt':
            doc = nlp_pt(page_text)
        if lang == 'en':
            doc = nlp_en(page_text)

        persons = []
        gpes = []
        #orgs = []
        for ent in doc.ents:
            if ent.label_ == "PER": #"PERSON":
                persons.append(process_uri(ent.text))
            if ent.label_ == "LOC": #"GPE":
                gpes.append(process_uri(ent.text))
            #if ent.label_ == "ORG":
            #    orgs.append(process_uri(ent.text))


            
        tripla = """
        stardog:""" + tese_uri + "__page_" + str(page_number + 1) + """ rdf:type bibo:Excerpt.
        stardog:""" + tese_uri +  """ <http://purl.obolibrary.org/obo/BFO_0000051> stardog:""" + tese_uri + "__page_" + str(page_number + 1 ) + """.
        stardog:""" + tese_uri + "__page_" + str(page_number + 1 ) + """ bibo:pages '""" + str(page_number + 1) +  """'. 
        """
        if lang == 'pt':
            tripla = tripla + "stardog:" + tese_uri + "__page_" + str(page_number + 1 ) + " <http://purl.org/dc/terms/description> '" + page_text.replace("'","").replace(u'\\', u' ') +  "'@pt."
        if lang == 'en':
            tripla = tripla + "stardog:" + tese_uri + "__page_" + str(page_number + 1 ) + " <http://purl.org/dc/terms/description> '" + page_text.replace("'","").replace(u'\\', u' ') +  "'@en."
        
        #print(set(persons), set(gpes))
        for per in set(persons):  
            tripla = tripla + """
            stardog:""" + per + """ rdf:type foaf:Person.
            stardog:""" + tese_uri + "__page_" + str(page_number + 1) + """ <https://schema.org/mentions> stardog:""" + per + """. 
            """

        for gpe in set(gpes): 
            tripla = tripla + """
            stardog:""" + gpe + """ rdf:type <https://schema.org/Place>.
            stardog:""" + tese_uri + "__page_" + str(page_number + 1) + """ <https://schema.org/mentions> stardog:""" + gpe + """. 
            """

        triplas = triplas + " " + tripla

    add_triplas_to_stardog(prefixos, triplas)
    return

In [1]:
#!python -m spacy download pt_core_news_lg
#!python -m spacy download en_core_web_lg
#%pip install -U scikit-learn

In [1]:
import os
import io
import stardog
import pandas as pd
import spacy
from spacy import displacy
from sklearn.cluster import DBSCAN
import numpy as np
import requests
from unidecode import unidecode
import re
import json
from langdetect import detect

### Specify Stardog connection details